In [1]:
import os

In [2]:
pwd

'd:\\chicken-disease-classification-deep-learning-project\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'd:\\chicken-disease-classification-deep-learning-project'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [10]:
from urllib.parse import urlparse

In [20]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores1 = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores1)


In [21]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

def evaluate_on_test_split(model_path: str, data_dir: str, image_size: list, batch_size: int):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.3)

    # Load the validation split (30% of the data)
    val_gen = datagen.flow_from_directory(
        directory=data_dir,
        subset="validation",
        target_size=image_size[:-1],
        batch_size=32,
        shuffle=False
    )

    # Collect all images and labels
    x_data, y_data = [], []
    for i in range(len(val_gen)):
        x_batch, y_batch = val_gen[i]
        x_data.extend(x_batch)
        y_data.extend(y_batch)
        if len(x_data) >= val_gen.samples:
            break

    x_data = np.array(x_data)
    y_data = np.array(y_data)

    # Convert labels from one-hot to integers for stratification
    y_labels = np.argmax(y_data, axis=1)

    # Proper stratified split to ensure balanced test set
    x_test, _, y_test, _ = train_test_split(
        x_data, y_data, test_size=0.3, stratify=y_labels, random_state=42
    )

    # Load model and evaluate
    model = tf.keras.models.load_model(model_path)
    test_loss, test_acc = model.evaluate(x_test, y_test, batch_size=batch_size)

    print(f"\n🧪 Test Evaluation Results")
    print(f"-------------------------")
    print(f"Test Loss     : {test_loss:.4f}")
    print(f"Test Accuracy : {test_acc:.4f}")

    # Predictions
    y_pred_probs = model.predict(x_test, batch_size=batch_size)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Metrics
    cm = confusion_matrix(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

    print("\n📊 Confusion Matrix:")
    print(cm)

    print(f"\n🔹 Accuracy : {accuracy:.4f}")
    print(f"🔹 Precision: {precision:.4f}")
    print(f"🔹 Recall   : {recall:.4f}")
    print(f"🔹 F1 Score : {f1:.4f}")

    print("\n📋 Classification Report:")
    print(classification_report(y_true, y_pred, digits=4, zero_division=0))


In [22]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

evaluate_on_test_split(
    model_path="artifacts/training/model.h5",
    data_dir="artifacts/data_ingestion/Chicken-fecal-images",
    image_size=val_config.params_image_size,
    batch_size=val_config.params_batch_size
)


[2025-05-27 23:35:25,065: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-27 23:35:25,072: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-27 23:35:25,073: INFO: common: created directory at: artifacts]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 34s 4s/step - loss: 0.9579 - accuracy: 0.9224
[2025-05-27 23:35:59,812: INFO: common: json file saved at: scores.json]
Found 116 images belonging to 2 classes.
6/6 [==============================] - 26s 4s/step - loss: 0.9384 - accuracy: 0.9259

🧪 Test Evaluation Results
-------------------------
Test Loss     : 0.9384
Test Accuracy : 0.9259
6/6 [==============================] - 29s 5s/step

📊 Confusion Matrix:
[[34  6]
 [ 0 41]]

🔹 Accuracy : 0.9259
🔹 Precision: 0.9362
🔹 Recall   : 0.9250
🔹 F1 Score : 0.9254

📋 Classification Report:
              precision    recall  f1-score   support

           0     1.0000    0.8500    0.9189        40
           1     0.87